In [1]:
def HGCD(a, b):  # Calcula el Half-GCD de los polinomios a, b.
    K = a.parent()  # Anillo de polinomios de a y b.
    dega = a.degree() # Grados de a y b.
    degb = b.degree()
    m = (dega / 2).ceil()
    if degb < m:  # Si el grado de b es menor que m, devolvemos la identidad.
        return matrix(K, 2, [1, 0, 0, 1])
    a0 = a.shift(-m)  # Cocientes de a y b por x^m.
    b0 = b.shift(-m)
    R = HGCD(a0, b0)  # Calculamos el HGCD recursivamente.
    r11 = R[0][0]  # Guardamos las componentes de la matriz R.
    r12 = R[0][1]
    r21 = R[1][0]
    r22 = R[1][1]
    d = (r11 * r22 - r12 * r21)  #Producto de R^-1 por la columna (a, b)'.
    a1 = (d * (a * r22 - b * r12))
    b1 = (d * (b * r11 - a * r21))
    if b1.degree() < m:  # Si el grado de b1 es menor que m, devolvemos R.
        return R
    qt, d = a1.quo_rem(b1)  # Si no, hacemos un paso del algoritmo de Euclides.
    c = b1  
    l = c.degree()
    k = 2 * m - l  # Tomamos k para que la recursión funcione.
    c0 = c.shift(-k)
    d0 = d.shift(-k)
    S = HGCD(c0, d0)  # Calculamos HGCD recurisvamente.
    RM = matrix(K, 2, prod(R, matrix(K, 2, [qt, K(1), K(1), K(0)])))
    Q = matrix(K, 2, prod(RM, S))  # Matriz R * [qt, 1, 1, 0] * S.
    return Q


def MIGCD(f,g):  # Calcula el gcd(f,g) aprovechando el HGCD.
    a = f  # No queremos que modifique la entrada.
    b = g
    if a % b == 0:  # Si f es divisible por g, GCD(f,g) = f.
        return b
    R = HGCD(a, b)  # Matriz R del HGCD.
    b0 = R[0][0] * a + R[0][1] * b  # Multiplicamos R por la columna (a, b)'
    b1 = R[1][0] * a + R[1][1] * b
    rem = b0 % b1  # Resto del cociente de b1 entre b0.
    if rem == 0:  # Si b0 es divisible por b1, GCD(f,g) = f
        return b1
    else:  # Si no, calculamos recursivamente el GCD con el resto.
        return MIGCD(b1, rem)


def prod(A, B):  # Calcula el producto de dos matrices como lista.
    A = list(A)  # Si A y B son matrices, las pasamos a listas.
    B = list(B)
    return [(A[0][0]*B[0][0]+A[0][1]*B[1][0],A[0][0]*B[0][1]+A[0][1]*B[1][1]),
            (A[1][0]*B[0][0]+A[1][1]*B[1][0],A[1][0]*B[0][1]+A[1][1]*B[1][1])]


def euclid_gcd(f, g):  # Calcula gcd(f,g) con el algoritmo de Euclides.
    r0 = f
    r1 = g
    while r1 != 0:
        r0, r1 = r1, r0%r1
    return r0

In [2]:
K.<x>=PolynomialRing(QQ)
f = x^5+x^4+x^3+x^2+x+1
g = x^4-2*x^3+3*x^2-x-7
show("R = ",HGCD(f, g))

'R = ' [1/4*x^2 + 11/16*x + 13/16                     x + 3]
[             1/4*x - 1/16                         1]

In [36]:
%time print(MIGCD(f,g))
%time euclid_gcd(f,g).factor()
K.<x>=PolynomialRing(QQ)
f = K.random_element(20)
g = K.random_element(16)  # Si g tiene grado entre 14 y 19, no termina: FALSE 1. (Grado 20 no se contempla).
print("R = ",HGCD(f,g))
%time show(MIGCD(f,g))
%time show(euclid_gcd(f,g))

R =  [1983070727442053049310428914421513402362821696599913784321017963863826926848024286674935628285823101152957828877531/1688224091073459814377435584172396929474365498124547588343878586243083026493303439385427260882232628792235548929426960*x^10 + 114675470888159902031695709886171008596174303576581488317800827569507274436040396603079406185928970944208292757937030634872105581362586687225360271345733/303382333697373679926186744111476727720033153044934905120654787985395826224563284733266769045111817160819853938993695212903002825897887787585966392177746640*x^9 + 336330222186973518997538278880645087023346905710164043206158998465545987166234438076981639561863623739251208693333179765437200094322076810660949960377731/303382333697373679926186744111476727720033153044934905120654787985395826224563284733266769045111817160819853938993695212903002825897887787585966392177746640*x^8 + 5485766554550634710260690291817764569685488941128360799969305702050337551823731826991773397410991744443803460788548725

1

CPU times: user 4.09 ms, sys: 2 µs, total: 4.09 ms
Wall time: 3.45 ms


230247728638149512515615155739924770361325348002343890094520411247301614524657992995706750084525872661518069624797747591581389877720651153740901457980981780978974251825789817457235708291688119196359826827708628630855248992214441957610806909847710946843828649288199432268200731543025626830727708508238588069305087881532672986521586343262985501738056143291759013602064535566113348272408536596359313935287377623387631877974460288428381167769184199810653690510209974286395110538245619714888455133613749642209693139677125136637335308516686561067632018378845565155535322929574558415112606448511520414941652801031770713799456037395590857781209110063197092928711849989546872276673396108174685789284883975099098505904059713657951677279836343571800534617704949432389814627157052621499324589405116170678329792829551747917617232504836813974379576140863996178891185471786951441993854301492789505047133461897257502571991943445685624158142312/161124440867009986240807113414714101226655590914802363715031901961140502471693845275867389348455525674121568711535978252639751382515166031845507065142965964575183926635042254179928182480572552704730942597269930407934292179044864107314355423548433463733258318971932427202284799037318453290069356287746304512180725972447452652499387749436397757280798524039523356268334690252137849315847458190842714633468104974432225231706599371235490817037312002047271068956527083727187336117330606809618035426171385013659870069916568154310370059284847378185776798437232247244056181404674437779177538297353142324036877169583459091710439670719227053196680100682307965099227307501684883883743933107585025361413716388319518957179491612664697064651417423173806102057554024334076860135704045552011420189506408539819705060428870535276405269117101529885232263447563865689320003970735715794744324212700473978546772923817295707518778475940203577235

CPU times: user 2.67 ms, sys: 0 ns, total: 2.67 ms
Wall time: 2.41 ms


In [32]:
K.<x>=PolynomialRing(QQ)
f=K.random_element(9)
g=K.random_element(8)
%time show(HGCD(f,g))

[-140277157099935/2982046432149987188*x^4 - 789379275909127271632479144725/1239316025944876760463706822638576*x^3 - 53573962152540637200830269299625/309829006486219190115926705659644*x^2 + 55370866455557322952613098393175/667324013970318255634303673728464*x + 1073320169262225721316592303597599/1084401522701767165405743469808754                                                                                                                                                                                                        68561045/6116146*x^3 + 1456899443983060/9351810473329*x^2 + 772942301531807771/18703620946658*x - 38505114627981148/9351810473329]
[                                                                                 140277157099935/1491023216074993594*x^3 + 847677405339189957265116842345/619658012972438380231853411319288*x^2 - 34973679239066006527200011715/29507524427258970487231114824728*x - 34910180183822496152249053854565/4337606090807068661622973879235016                                                                                                                                                                                                                                                 -68561045/3058073*x^2 - 3123463568532405/9351810473329*x + 322547036613179/9351810473329]

CPU times: user 2.85 ms, sys: 95 µs, total: 2.94 ms
Wall time: 2.2 ms


In [31]:
K.<x>=PolynomialRing(QQ)
f = K(7 * (x - 3 / 2) * (x - 1) * (x ** 4+ 3 * x + 2))
g = K(6* x ** 3 - 17 * x ** 2 + 14 * x - 3)
show(HGCD(f,g))
%time print(MIGCD(f,g).factor())
%time print(euclid_gcd(f,g).factor())

[7/6*x^3 + 7/18*x^2 + 7/54*x + 287/81                                    1]
[                                   1                                    0]

(x - 3/2) * (x - 1)
CPU times: user 822 µs, sys: 27 µs, total: 849 µs
Wall time: 829 µs
(1708/81) * (x - 3/2) * (x - 1)
CPU times: user 312 µs, sys: 10 µs, total: 322 µs
Wall time: 322 µs


In [6]:
K.<x>=PolynomialRing(GF(101))
f = K(14 * (2 * x - 3) * (x - 1) * (x ** 4 + 3 * x + 2))
g = K(6* x ** 3 - 17 * x ** 2 + 14 * x - 3)
%time show(MIGCD(f,g).factor())
%time show(euclid_gcd(f,g).factor())

(x + 49) * (x + 100)

CPU times: user 5.66 ms, sys: 3.18 ms, total: 8.84 ms
Wall time: 8.78 ms


(22) * (x + 49) * (x + 100)

CPU times: user 1.13 ms, sys: 138 µs, total: 1.27 ms
Wall time: 1.04 ms


In [7]:
K.<x>=PolynomialRing(GF(101))
f = x^5+x^4+x^3+x^2+x+1
g = x^4-2*x^3+3*x^2-x-7
%time show(MIGCD(f,g))
%time show(euclid_gcd(f,g).factor())

x + 1

CPU times: user 1.58 ms, sys: 194 µs, total: 1.78 ms
Wall time: 1.3 ms


(13) * (x + 1)

CPU times: user 1.8 ms, sys: 0 ns, total: 1.8 ms
Wall time: 1.24 ms


In [8]:
K.<x>=PolynomialRing(GF(101))
f = K.random_element(99)
g = K.random_element(45)
%time print(MIGCD(f,g))
%time print(euclid_gcd(f,g))

1
CPU times: user 654 µs, sys: 0 ns, total: 654 µs
Wall time: 525 µs
42
CPU times: user 149 µs, sys: 0 ns, total: 149 µs
Wall time: 144 µs


In [9]:
K.<x>=PolynomialRing(GF(101))
f = K.random_element(99)
g = K.random_element(55)
HGCD(f,g)
#%time MIGCD(f,g)
#%time euclid_gcd(f,g)

FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1


FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1
FALSE 1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-2386587fd393>", line 4, in <module>
    HGCD(f,g)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c737e18233b>", line 10, in HGCD
    R = HGCD(a0, b0)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c737e18233b>", line 10, in HGCD
    R = HGCD(a0, b0)
  File "<ipython-input-1-8c737e18233b>", line 30, in HGCD
    S = HGCD(c0, d0)
  File "<ipython-input-1-8c7

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
def MIGCD2(f,g):
    K = f.parent()
    E = matrix(K, 2, 2, [1, 0, 0, 1])  # En este caso, no debe devolver esta matriz, sino la matriz de polinomios.
    m = (f.degree() / 2).ceil()
    a = f
    b = g
    # Queremos que en todo el algoritmo se cumpla vectorI[a, b]) == E*vector([f, g]).
    while b != 0:
        if b.degree() < m:
            q, r = a.quo_rem(b)
            a, b = b, r
            E = matrix(K, 2, prod(matrix(K, 2, 2, [0, 1, 1, -q]), E))  # Si usamos matrices, es costosos. Es mejor trabajar con listas.
        else:
            R = HGCD(a,b)
            a = R[0][0] * a + R[0][1] * b
            b = R[1][0] * a + R[1][1] * b
            E = prod(R, E) # Para que siga dándose la igualdad.
    return a, E[0][0], E[0][1]

In [17]:
A = matrix(K, 2, [1, 1, 1, 2])
B = matrix(K, 2, [2, 1, 3, 2])

[5 3]
[8 5]